<center>
    <b><h1> Couserwork Assignment: Text Classification</h1></b>
</center>

<center>
    <h3>University of London</h3>
</center>

<center>
    <h3>BSc in Computer Science</h3>
</center>

<center>
    <h3>Natural Language Processing</h3>
</center>

<div style="page-break-after: always;"></div>

<h1> Table of Content</h1>

<style>
ul {
    font-size: 20px;
}

ul ul {
    font-size: 15px;
}
</style>

<ul>
    <li>
        <a href="#intro">1. Introduction</a>
        <ul>
            <li><a href="#1.1">1.1 Problem Area</a></li>
            <li><a href="#1.2">1.2 Objectives</a></li>
        </ul>
    </li>
    <li>
        <a href="#section2">Section 2</a>
        <ul>
            <li><a href="#subsection2.1">Subsection 2.1</a></li>
            <li><a href="#subsection2.2">Subsection 2.2</a></li>
        </ul>
    </li>
    <li>
        <a href="#section3">Section 3</a>
        <ul>
            <li><a href="#subsection3.1">Subsection 3.1</a></li>
            <li><a href="#subsection3.2">Subsection 3.2</a></li>
        </ul>
    </li>
</ul>


<div style="page-break-after: always;"></div>

<h2 id="intro">1. Introduction</h2>
<h3 id="1.1">1.1 Problem area</h3>

<h3 id="">Data Extraction</h3>

In [1]:
# Import library needed for data analysis
import os
import random
import pandas as pd

In [2]:
# Path to the folders
pos_folder = r'C:\Users\xjie\Documents\SIM\Y3S2\nlp\aclImdb\train\pos'
neg_folder = r'C:\Users\xjie\Documents\SIM\Y3S2\nlp\aclImdb\train\neg'

In [3]:
def count_files(folder_path):
    if not os.path.isdir(folder_path):
        print(f"Error: {folder_path} is not a valid directory.")
        return

    file_count = 0

    for _, _, files in os.walk(folder_path):
        file_count += len(files)

    return file_count

In [4]:
# Example the positive training data file
file_count = count_files(pos_folder)
print(f"The number of files in positive training folder is: {file_count}")

The number of files in positive training folder is: 12500


In [5]:
# Example the positive training data file
file_count = count_files(pos_folder)
print(f"The number of files in positive training folder is: {file_count}")

The number of files in positive training folder is: 12500


As the training data set is too big for the computer, only 2000 data from positive and negative folder will be selected to make the program runs faster.

In [6]:
# Function to extract unique ID and rating from file name
def extract_info(file_name):
    id_rating = file_name.split('.')[0]
    unique_id, rating = id_rating.split('_')
    return int(unique_id), int(rating)

In [7]:
# Extracting training data
# Initialize an empty DataFrame
data = pd.DataFrame(columns=['Sentiment', 'Unique ID', 'Rating', 'Content'])

# Process positive reviews
pos_files = random.sample(os.listdir(pos_folder), 2000)
for file_name in pos_files:
    file_path = os.path.join(pos_folder, file_name)
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    unique_id, rating = extract_info(file_name)
    pos_data = pd.DataFrame({'Sentiment': ['pos'], 'Unique ID': [unique_id], 'Rating': [rating], 'Content': [content]})
    data = pd.concat([data, pos_data], ignore_index=True)

# Process negative reviews
neg_files = random.sample(os.listdir(neg_folder), 2000)
for file_name in neg_files:
    file_path = os.path.join(neg_folder, file_name)
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    unique_id, rating = extract_info(file_name)
    neg_data = pd.DataFrame({'Sentiment': ['neg'], 'Unique ID': [unique_id], 'Rating': [rating], 'Content': [content]})
    data = pd.concat([data, neg_data], ignore_index=True)

# Print the resulting DataFrame
data.head()

,Sentiment,Unique ID,Rating,Content
0,pos,4801,7,I watched this with my whole family as a 9 yea...
1,pos,3551,8,This is a Frank Loesser masterpiece of amusing...
2,pos,4660,7,"""Hot Millions"" is a well-written, well-acted t..."
3,pos,4180,10,I was overtaken by the emotion. Unforgettable ...
4,pos,10386,8,"what is wrong with you people, if you weren't ..."


<h3 id="">Prelimary analysis and data cleaning</h3>

In [8]:
# Examining the data
data.describe()

,Sentiment,Unique ID,Rating,Content
count,4000,4000,4000,4000
unique,2,3653,8,3994
top,pos,9752,1,Bette Midler is again Divine! Raunchily humoro...
freq,2000,2,827,2


It was noted that both Unique ID and Content has duplicated values. Thus, an algorithm will be implemented to check for these values and handle them. 

As a <b>new ID is assigned</b> as the index for the data frame, there is <b>no need</b> for the Unique ID from positive sentiment and negative sentiment folder. Thus, the <b>unique ID column will be dropped</b>. 

In [9]:
# Removing the unique ID
data.drop("Unique ID",axis=1,inplace=True)
# Examining the data again
data.describe()

,Sentiment,Rating,Content
count,4000,4000,4000
unique,2,8,3994
top,pos,1,Bette Midler is again Divine! Raunchily humoro...
freq,2000,827,2


In [10]:
# Checking for duplicates 
duplicates = data[data['Content'].duplicated(keep=False)]
duplicates_sorted = duplicates.sort_values(by='Content')

if duplicates_sorted.empty:
    print("No repeated data entry found.")
else:
    print("Repeated data entry/entries found:")
    duplicates_pairs = duplicates_sorted.groupby('Content').apply(lambda x: x.reset_index(drop=True))
    duplicates_pairs.reset_index(drop=True, inplace=True)

Repeated data entry/entries found:


In [11]:
removed_data = duplicates_pairs.drop_duplicates(subset ="Content", keep='first')

In [12]:
removed_data

,Sentiment,Rating,Content
0,pos,10,Bette Midler is again Divine! Raunchily humoro...
2,neg,1,How has this piece of crap stayed on TV this l...
4,neg,1,"I absolutely hate this programme, what kind of..."
6,neg,4,The 60s (1999) D: Mark Piznarski. Josh Hamilto...
8,neg,1,This movie is a disgrace to the Major League F...
10,pos,7,"While there aren't any talking animals, big la..."


In [13]:
# Add the removed duplicates such that the data is balanced
# Count the number of pos and neg entries in 'removed_data'
pos_count = removed_data[removed_data['Sentiment'] == 'pos'].shape[0]
neg_count = removed_data[removed_data['Sentiment'] == 'neg'].shape[0]

# Initialize an empty DataFrame to store the added data
added_data = pd.DataFrame(columns=['Sentiment', 'Unique_ID', 'Rating', 'Content'])

# Load data from the pos_folder
pos_files = os.listdir(pos_folder)
pos_files = [filename for filename in pos_files if filename.endswith('.txt')]

for filename in pos_files[:pos_count]:
    file_path = os.path.join(pos_folder, filename)
    unique_id, rating = filename[:-4].split('_')
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    row = pd.DataFrame({
        'Sentiment': ['pos'],
        'Unique_ID': [unique_id],
        'Rating': [rating],
        'Content': [content]
    })
    added_data = pd.concat([added_data, row], ignore_index=True)

# Load data from the neg_folder
neg_files = os.listdir(neg_folder)
neg_files = [filename for filename in neg_files if filename.endswith('.txt')]

for filename in neg_files[:neg_count]:
    file_path = os.path.join(neg_folder, filename)
    unique_id, rating = filename[:-4].split('_')
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    row = pd.DataFrame({
        'Sentiment': ['neg'],
        'Unique_ID': [unique_id],
        'Rating': [rating],
        'Content': [content]
    })
    added_data = pd.concat([added_data, row], ignore_index=True)

,Sentiment,Unique_ID,Rating,Content
0,pos,0,9,Bromwell High is a cartoon comedy. It ran at t...
1,pos,10000,8,Homelessness (or Houselessness as George Carli...
2,neg,0,3,Story of a man who has unnatural feelings for ...
3,neg,10000,4,Airport '77 starts as a brand new luxury 747 p...
4,neg,10001,4,This film lacked something I couldn't put my f...
5,neg,10002,1,"Sorry everyone,,, I know this is supposed to b..."


In [16]:
# Print the added_data DataFrame
# Removing the unique ID
added_data.drop("Unique_ID",axis=1,inplace=True)
added_data

,Sentiment,Rating,Content
0,pos,9,Bromwell High is a cartoon comedy. It ran at t...
1,pos,8,Homelessness (or Houselessness as George Carli...
2,neg,3,Story of a man who has unnatural feelings for ...
3,neg,4,Airport '77 starts as a brand new luxury 747 p...
4,neg,4,This film lacked something I couldn't put my f...
5,neg,1,"Sorry everyone,,, I know this is supposed to b..."


In [20]:
# Remove non-unique duplicates
test=data.drop_duplicates(subset='Content', keep='first')
len(test)
new=pd.concat([added_data, test], ignore_index =  True)

In [21]:
new

,Sentiment,Rating,Content
0,pos,9,Bromwell High is a cartoon comedy. It ran at t...
1,pos,8,Homelessness (or Houselessness as George Carli...
2,neg,3,Story of a man who has unnatural feelings for ...
3,neg,4,Airport '77 starts as a brand new luxury 747 p...
4,neg,4,This film lacked something I couldn't put my f...
...,...,...,...
3995,neg,4,"I confess--Emma, in my opinion, is the single ..."
3996,neg,3,Because others have gone to the trouble of sum...
3997,neg,2,This movie has some of the most awesome cars I...
3998,neg,1,"""Why?""<br /><br />That simple question had to ..."


In [22]:
# Checking for duplicates 
duplicates = new[new['Content'].duplicated(keep=False)]
duplicates_sorted = duplicates.sort_values(by='Content')

if duplicates_sorted.empty:
    print("No repeated data entry found.")
else:
    print("Repeated data entry/entries found:")
    duplicates_pairs = duplicates_sorted.groupby('Content').apply(lambda x: x.reset_index(drop=True))
    duplicates_pairs.reset_index(drop=True, inplace=True)

No repeated data entry found.


In [ ]:
# check for null data
data.isnull().sum()

In [ ]:
# double checking the total number of data entries
data['Sentiment'].value_counts()